# 1.- Checar la red de contenedores para conocer la ip donde se va a realizar la petición a la API

## Ejemplo

In [4]:
#docker network ls
#docker network inspect b5801a78ce54

In [11]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import requests
import warnings
warnings.filterwarnings('ignore')

# 2.- Request de la API

In [4]:
retorno = requests.get('http://172.18.0.7:8000/catalog')
print(retorno)

<Response [200]>


In [5]:
print(type(retorno.json()))

<class 'dict'>


In [6]:
retorno.json()

{'items': [{'id': 1,
   'date': '13/2/2020',
   'customer': 6,
   'products': [{'product': 54, 'quantity': 8},
    {'product': 82, 'quantity': 7}]},
  {'id': 2,
   'date': '10/8/2021',
   'customer': 11,
   'products': [{'product': 47, 'quantity': 1},
    {'product': 82, 'quantity': 5},
    {'product': 56, 'quantity': 6},
    {'product': 48, 'quantity': 8},
    {'product': 20, 'quantity': 2}]},
  {'id': 3,
   'date': '4/8/2020',
   'customer': 11,
   'products': [{'product': 76, 'quantity': 8},
    {'product': 49, 'quantity': 1}]},
  {'id': 4,
   'date': '20/6/2021',
   'customer': 19,
   'products': [{'product': 7, 'quantity': 4}, {'product': 1, 'quantity': 1}]},
  {'id': 5,
   'date': '1/8/2020',
   'customer': 7,
   'products': [{'product': 61, 'quantity': 2},
    {'product': 16, 'quantity': 10}]},
  {'id': 6,
   'date': '25/7/2020',
   'customer': 3,
   'products': [{'product': 67, 'quantity': 5}]},
  {'id': 7,
   'date': '26/8/2020',
   'customer': 13,
   'products': [{'product': 

# 3.- Extracción desde pyspark

In [12]:
spark = SparkSession.builder.appName("Engie").getOrCreate()

In [13]:
df_spark = spark.createDataFrame(retorno.json()['items'])

In [14]:
df_spark.show(7)

+--------+---------+---+--------------------+
|customer|     date| id|            products|
+--------+---------+---+--------------------+
|       6|13/2/2020|  1|[{product -> 54, ...|
|      11|10/8/2021|  2|[{product -> 47, ...|
|      11| 4/8/2020|  3|[{product -> 76, ...|
|      19|20/6/2021|  4|[{product -> 7, q...|
|       7| 1/8/2020|  5|[{product -> 61, ...|
|       3|25/7/2020|  6|[{product -> 67, ...|
|      13|26/8/2020|  7|[{product -> 79, ...|
+--------+---------+---+--------------------+
only showing top 7 rows



In [15]:
df_spark.printSchema()

root
 |-- customer: long (nullable = true)
 |-- date: string (nullable = true)
 |-- id: long (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: long (valueContainsNull = true)



### Abriendo Modelo No Relacional

In [16]:
(df_spark
 .select("*",F.explode("products").alias("element"))
 .select("*",F.explode("element")).drop("products","element")
 .withColumnRenamed("key","columna_1").withColumnRenamed("value","columna_2").show()
)

+--------+---------+---+---------+---------+
|customer|     date| id|columna_1|columna_2|
+--------+---------+---+---------+---------+
|       6|13/2/2020|  1|  product|       54|
|       6|13/2/2020|  1| quantity|        8|
|       6|13/2/2020|  1|  product|       82|
|       6|13/2/2020|  1| quantity|        7|
|      11|10/8/2021|  2|  product|       47|
|      11|10/8/2021|  2| quantity|        1|
|      11|10/8/2021|  2|  product|       82|
|      11|10/8/2021|  2| quantity|        5|
|      11|10/8/2021|  2|  product|       56|
|      11|10/8/2021|  2| quantity|        6|
|      11|10/8/2021|  2|  product|       48|
|      11|10/8/2021|  2| quantity|        8|
|      11|10/8/2021|  2|  product|       20|
|      11|10/8/2021|  2| quantity|        2|
|      11| 4/8/2020|  3|  product|       76|
|      11| 4/8/2020|  3| quantity|        8|
|      11| 4/8/2020|  3|  product|       49|
|      11| 4/8/2020|  3| quantity|        1|
|      19|20/6/2021|  4|  product|        7|
|      19|